# Parseado de datos para Discriminador

Este notebook genera textos de control negativo (no Shakespeare) a partir de obras literarias de dominio público obtenidas de Project Gutenberg. <br>
Los textos se limpian, segmentan en chunks de longitud comparable a los textos generados por modelos, y se guardan como archivos .txt para ser utilizados en el entrenamiento y evaluación de un clasificador estilístico.

## 1. Imports.

In [ ]:
import os
import re
import random
from pathlib import Path

## 2. Configuración.

In [ ]:
# Directorios de trabajo (pensado para Google Colab)
RAW_DIR = Path("/content/raw_fino")
OUTPUT_DIR = Path(
    "/content/drive/MyDrive/StoryWriter/Data/Training_data/"
    "clasificator_train/negativos_finos"
)

RAW_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros de longitud de texto
MIN_WORDS = 150
MAX_WORDS = 400

# Máximo de chunks por libro (para balancear el dataset)
MAX_CHUNKS_PER_BOOK = 100

# Semilla para reproducibilidad
random.seed(42)

## 3. Descarga de libros (control negativo grueso).

In [ ]:
GUTENBERG_URLS = [
    # Aventura / General
    "https://www.gutenberg.org/cache/epub/2701/pg2701.txt",  # Moby Dick
    "https://www.gutenberg.org/cache/epub/74/pg74.txt",     # Tom Sawyer
    "https://www.gutenberg.org/cache/epub/55/pg55.txt",     # Wizard of Oz

    # Ciencia ficción
    "https://www.gutenberg.org/cache/epub/18857/pg18857.txt",  # Journey to the Centre
    "https://www.gutenberg.org/cache/epub/35/pg35.txt",       # The Time Machine
    "https://www.gutenberg.org/cache/epub/36/pg36.txt",       # War of the Worlds

    # Fantasía / Romance
    "https://www.gutenberg.org/cache/epub/11/pg11.txt",    # Alice
    "https://www.gutenberg.org/cache/epub/45/pg45.txt",    # Anne of Green Gables
    "https://www.gutenberg.org/cache/epub/16/pg16.txt",    # Peter Pan

    # Terror / Drama
    "https://www.gutenberg.org/cache/epub/345/pg345.txt",  # Dracula
    "https://www.gutenberg.org/cache/epub/84/pg84.txt",    # Frankenstein
    "https://www.gutenberg.org/cache/epub/43/pg43.txt",    # Jekyll & Hyde
]

for url in GUTENBERG_URLS:
    fname = url.split("/")[-1]
    out_path = RAW_DIR / fname
    if out_path.exists():
        print("Ya existe:", out_path.name)
        continue
    !wget -q "{url}" -O "{out_path}"
    print("Descargado:", out_path.name)

## 4. Definición de funciones.

In [ ]:
def clean_gutenberg_text(text: str) -> str:
    """
    Limpieza mínima del texto:
    - normaliza saltos de línea
    - elimina espacios duplicados
    - colapsa bloques grandes de líneas vacías
    """
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def split_into_paragraphs(text: str):
    """
    Divide el texto en párrafos usando dobles saltos de línea.
    """
    return [p.strip() for p in text.split("\n\n") if p.strip()]

def make_chunks_from_paragraphs(paragraphs, min_words=MIN_WORDS, max_words=MAX_WORDS):
    """
    Junta párrafos hasta alcanzar un rango de palabras [min_words, max_words].
    Devuelve una lista de strings (chunks).
    """
    chunks = []
    current_words = []
    current_len = 0

    for para in paragraphs:
        words = para.split()
        if current_len + len(words) > max_words and current_words:
            if current_len >= min_words:
                chunks.append(" ".join(current_words))
            current_words = []
            current_len = 0

        current_words.extend(words)
        current_len += len(words)

    if current_words and current_len >= min_words:
        chunks.append(" ".join(current_words))

    return chunks

## 5. Procesamiento completo del corpus.

In [ ]:
def process_books(input_dir: Path):
    """
    Procesa todos los libros del directorio:
    - limpia texto
    - divide en párrafos
    - genera chunks
    - selecciona un subconjunto aleatorio
    """
    all_chunks = []

    for path in sorted(input_dir.glob("*.txt")):
        print("Procesando:", path.name)
        text = path.read_text(encoding="utf-8", errors="ignore")
        clean = clean_gutenberg_text(text)

        paragraphs = split_into_paragraphs(clean)
        chunks = make_chunks_from_paragraphs(paragraphs)

        random.shuffle(chunks)
        selected = chunks[:MAX_CHUNKS_PER_BOOK]
        all_chunks.extend(selected)

    return all_chunks


chunks = process_books(RAW_DIR)
print(f"Total de chunks generados: {len(chunks)}")

## 6. Guardado.

In [ ]:
for i, chunk in enumerate(chunks):
    out_path = OUTPUT_DIR / f"chunk_{i:05d}.txt"
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(chunk)

print(f"Guardados {len(chunks)} archivos en:")
print(OUTPUT_DIR)